In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd /gdrive/MyDrive/caltech-101/

/gdrive/MyDrive/caltech-101


In [ ]:
import cv2
import numpy as np
import os
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
image_paths = []
for class_name in os.listdir("./data"):
    class_path = os.path.join("./data", class_name)
    for image_name in os.listdir(class_path):
        image_paths.append((os.path.join(class_path, image_name), class_name))

In [ ]:
train_paths, test_paths = train_test_split(image_paths, test_size=0.05, random_state=42)

In [ ]:
print(len(train_paths))
print(len(test_paths))

1150
61


In [ ]:
def get_descriptors(sift, img):
    kp, des = sift.detectAndCompute(img, None)
    return des

def read_image(img_path):
    img = cv2.imread(img_path, 0)
    return cv2.resize(img, (150, 150))

def extract_features(image_paths, sift):
    descriptor_list = []
    labels = []
    for img_path, label in image_paths:
        img = read_image(img_path)
        des = get_descriptors(sift, img)
        if des is not None:
            descriptor_list.append(des)
            labels.append(label)
    return descriptor_list, labels

def stack_descriptors(descriptor_list):
    descriptors = np.array(descriptor_list[0])
    for descriptor in descriptor_list[1:]:
        descriptors = np.vstack((descriptors, descriptor))
    return descriptors

def extract_image_features(descriptor_list, kmeans, num_clusters):
    image_features = np.zeros((len(descriptor_list), num_clusters))
    for i, descriptors in enumerate(descriptor_list):
        for descriptor in descriptors:
            idx = kmeans.predict(descriptor.reshape(1, -1))
            image_features[i][idx] += 1
    return image_features

def train_svm(X_train, y_train, kernel='linear'):
    svm = SVC(kernel=kernel)
    svm.fit(X_train, y_train)
    return svm

In [ ]:
nfeatures_values = [200, 300, 400]
sigma_values = [0.7, 0.9, 1.1]
num_clusters_values = [200, 400, 600]

best_accuracy = 0
best_params = None

for nfeatures in nfeatures_values:
    for sigma in sigma_values:
        for num_clusters in num_clusters_values:
            print(f"nfeatures: {nfeatures}, sigma: {sigma}, num_clusters: {num_clusters}")

            extractor = cv2.SIFT_create(nfeatures=nfeatures, sigma=sigma)
            train_descriptors, train_labels = extract_features(train_paths, extractor)
            train_descriptors_stacked = stack_descriptors(train_descriptors)

            kmeans = KMeans(n_clusters=num_clusters).fit(train_descriptors_stacked)

            train_features = extract_image_features(train_descriptors, kmeans, num_clusters)
            scaler = StandardScaler()
            train_features_normalized = scaler.fit_transform(train_features)

            svm = train_svm(train_features_normalized, train_labels)

            test_descriptors, test_labels = extract_features(test_paths, extractor)
            test_features = extract_image_features(test_descriptors, kmeans, num_clusters)
            test_features_normalized = scaler.transform(test_features)

            predictions = svm.predict(test_features_normalized)

            accuracy = np.mean(predictions == test_labels)
            print("Accuracy:", accuracy)

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {'nfeatures': nfeatures, 'sigma': sigma, 'num_clusters': num_clusters}

print("\nBest Model:")
print("Accuracy:", best_accuracy)
print("Best Parameters:", best_params)

nfeatures: 200, sigma: 0.7, num_clusters: 200


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy: 0.5901639344262295
nfeatures: 200, sigma: 0.7, num_clusters: 400


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy: 0.7213114754098361
nfeatures: 200, sigma: 0.7, num_clusters: 600


...

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy: 0.6885245901639344
nfeatures: 300, sigma: 0.9, num_clusters: 600


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy: 0.7213114754098361
nfeatures: 300, sigma: 1.1, num_clusters: 200


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
extractor = cv2.SIFT_create(nfeatures=300, nOctaveLayers=8, contrastThreshold=0.03, edgeThreshold=20.0, sigma=0.7)

# Extract features from train set
train_descriptors, train_labels = extract_features(train_paths, extractor)

In [ ]:
def stack_descriptors(descriptor_list):
    descriptors = np.array(descriptor_list[0])
    for descriptor in descriptor_list[1:]:
        descriptors = np.vstack((descriptors, descriptor))
    return descriptors
train_descriptors_stacked = stack_descriptors(train_descriptors)

In [ ]:
num_clusters = 250
kmeans = KMeans(n_clusters=num_clusters).fit(train_descriptors_stacked)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
def extract_image_features(descriptor_list, kmeans, num_clusters):
    image_features = np.zeros((len(descriptor_list), num_clusters))
    for i, descriptors in enumerate(descriptor_list):
        for descriptor in descriptors:
            idx = kmeans.predict(descriptor.reshape(1, -1))
            image_features[i][idx] += 1
    return image_features
# Extract image features
train_features = extract_image_features(train_descriptors, kmeans, num_clusters)

# Normalize features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)

In [ ]:
def train_svm(X_train, y_train, kernel='linear'):
    svm = SVC(kernel=kernel)
    svm.fit(X_train, y_train)
    return svm
svm = train_svm(train_features_normalized, train_labels)

In [ ]:
# Extract features from test set
test_descriptors, test_labels = extract_features(test_paths, extractor)
test_features = extract_image_features(test_descriptors, kmeans, num_clusters)
test_features_normalized = scaler.transform(test_features)

# Test SVM
predictions = svm.predict(test_features_normalized)

# Evaluate
accuracy = np.mean(predictions == test_labels)
print("Accuracy:", accuracy)

In [ ]:
num_clusters = 400
kmeans = KMeans(n_clusters=num_clusters).fit(train_descriptors_stacked)

# Extract image features
train_features = extract_image_features(train_descriptors, kmeans, num_clusters)

# Normalize features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)

svm = train_svm(train_features_normalized, train_labels)

# Extract features from test set
test_descriptors, test_labels = extract_features(test_paths, extractor)
test_features = extract_image_features(test_descriptors, kmeans, num_clusters)
test_features_normalized = scaler.transform(test_features)

# Test SVM
predictions = svm.predict(test_features_normalized)

# Evaluate
accuracy = np.mean(predictions == test_labels)
print("Accuracy:", accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy: 0.64


In [ ]:
maxlen = 0
for d in train_descriptors:
  if maxlen < len(d):
    maxlen = len(d)
print(maxlen)

303
